In [ ]:
!pip install langchain openai google-search-results beautifulsoup4 requests

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=ebff90f5c4bda2d905a996c3ab037b2a8168665e06f3deff6be61a084544413f
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.tools import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.utilities import SerpAPIWrapper


In [ ]:
from google.colab import userdata
serpapi_api_key = userdata.get('serpapi')
openai_api_key = userdata.get('openai-api-key')

# 환경 변수 설정 (API 키 필요)
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPAPI_API_KEY"] = serpapi_api_key

In [ ]:

# OpenAI 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.6)

url = 'http://rule.kbu.ac.kr/lmxsrv/main/main.do'

In [ ]:

# 검색 함수 정의
def search_google(query):
    search = SerpAPIWrapper()
    results = search.run(query)
    return results

# 웹페이지 스크래핑 함수 정의
def scrape_url(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "페이지를 불러오지 못했습니다."

    soup = BeautifulSoup(response.text, "html.parser")
    paragraphs = soup.find_all("p")
    text = "\n".join([p.get_text() for p in paragraphs[:5]])  # 상위 5개 문단 가져오기
    return text if text else "유효한 텍스트를 찾지 못했습니다."

# LangChain 도구 등록
search_tool = Tool(
    name="Google Search",
    func=search_google,
    description="사용자의 질문을 기반으로 관련 웹페이지를 검색합니다."
)

scrape_tool = Tool(
    name="Web Scraper",
    func=scrape_url,
    description="주어진 URL에서 주요 내용을 추출합니다."
)

# 에이전트 설정
agent = initialize_agent(
    tools=[search_tool, scrape_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 사용자 질문 처리
user_query = "해당 사이트에서 수강 최소 학점이 무엇인지 자세히 알려줘"
response = agent.run(user_query)
print(response)



> Entering new AgentExecutor chain...
해당 사이트에서 수강 최소 학점에 대한 정보를 찾기 위해, 관련된 웹페이지를 검색해야겠다.  
Action: Google Search  
Action Input: "해당 사이트 수강 최소 학점"  
Observation: ['일단, 미국 교환학생 즉 J1 비자 학생은 최소 12학점을 들어야해요! 몇 학점을 수강신청할지는 교환학생을 가는 목적에 따라 달라질 수 있는데요,. 저의 ...', '수강신청학점 : 최소 1학점 ~ 최대 19학점. ※다음의 경우, 추가 3학점까지 초과하여 수강신청을 할 수 있습니다. 1) F등급 없이 전체 성적평점평균이 3.75 이상인 학생.', '연세포탈시스템 수강신청 사이트에서 수강신청 : 오전 7시부터 로그인 가능, ... 철회 후 최소 수강학점은 1학점 이상이어야 함. 04. Pass/Non Pass 평가과목 역사 ...', 'A) 클래스넷 → 수강신청 안내 → 개인별 수강가능학점 조회바로가기 에서 본인이 수강할 수 있는 최대 학점을 확인할 수 있습니다. 예시) 클래스넷-수강 ...', '수강신청(受講申請)은 대학에서 매 학기가 시작하기 전에 자신이 들을 과목을 정하여 신청하는 것을 말한다. 대다수는 개강 2~3주 전에 수강신청을 하는 것이 보통이나 ...', '... 최소 수강학점은 3학점 이상으로 하며, 반드시 정규등록을 필하여야 함. 아래 해당 소속 신입생의 경우 1학년 동안 2학점 이내 초과 수강신청 가능. 2019학년도 신입생 ...', '최대학점: 19학점 ※ 수강신청 학점은 최대 24학점을 초과할 수 없음(초과하여 중복인정 불가). · 최소학점: 12학점(단, 7학기 9학점, 8학기 6학점) · 성적우수 장학금 가능 ...', '수강신청 · 가.편입생은 편입학과 편입학년의 교육과정을 적용받으며 해당 학과의 졸업 이수 학점을 적용받습니다. · 나.편입 시 전적 대학에서 이수한 학점은 일반 선택 ...', '매학기 최소 수강신청 학점 수는 1학점 이상으

In [ ]:
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.4)

# 사용자 질문 처리
user_query = "해당 사이트에서 수강 최소 학점이 무엇인지 자세히 알려줘"
response = agent.run(user_query)
print(response)



> Entering new AgentExecutor chain...
해당 사이트에서 수강 최소 학점에 대한 정보를 찾기 위해 구글 검색을 진행하겠습니다. 
Action: Google Search
Action Input: "해당 사이트 수강 최소 학점"
Observation: ['일단, 미국 교환학생 즉 J1 비자 학생은 최소 12학점을 들어야해요! 몇 학점을 수강신청할지는 교환학생을 가는 목적에 따라 달라질 수 있는데요,. 저의 ...', '수강신청학점 : 최소 1학점 ~ 최대 19학점. ※다음의 경우, 추가 3학점까지 초과하여 수강신청을 할 수 있습니다. 1) F등급 없이 전체 성적평점평균이 3.75 이상인 학생.', '연세포탈시스템 수강신청 사이트에서 수강신청 : 오전 7시부터 로그인 가능, ... 철회 후 최소 수강학점은 1학점 이상이어야 함. 04. Pass/Non Pass 평가과목 역사 ...', 'A) 클래스넷 → 수강신청 안내 → 개인별 수강가능학점 조회바로가기 에서 본인이 수강할 수 있는 최대 학점을 확인할 수 있습니다. 예시) 클래스넷-수강 ...', '수강신청(受講申請)은 대학에서 매 학기가 시작하기 전에 자신이 들을 과목을 정하여 신청하는 것을 말한다. 대다수는 개강 2~3주 전에 수강신청을 하는 것이 보통이나 ...', '... 최소 수강학점은 3학점 이상으로 하며, 반드시 정규등록을 필하여야 함. 아래 해당 소속 신입생의 경우 1학년 동안 2학점 이내 초과 수강신청 가능. 2019학년도 신입생 ...', '최대학점: 19학점 ※ 수강신청 학점은 최대 24학점을 초과할 수 없음(초과하여 중복인정 불가). · 최소학점: 12학점(단, 7학기 9학점, 8학기 6학점) · 성적우수 장학금 가능 ...', '수강신청 · 가.편입생은 편입학과 편입학년의 교육과정을 적용받으며 해당 학과의 졸업 이수 학점을 적용받습니다. · 나.편입 시 전적 대학에서 이수한 학점은 일반 선택 ...', '매학기 최소 수강신청 학점 수는 1학점 이상으로 한다. 유급